In [1]:
pip install predictionguard langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from predictionguard import PredictionGuard
from getpass import getpass

In [3]:
# Prompt for API key securely
pg_access_token = getpass('Enter your Prediction Guard API key: ')
os.environ['PREDICTIONGUARD_API_KEY'] = pg_access_token

In [4]:
# Initialize Prediction Guard Client
client = PredictionGuard()

In [5]:
# Sample meeting transcription
tanscription_text = """
John: We need to finalize the Q1 budget by next Monday.
Sarah: I'll prepare the first draft by Friday for review.
Mike: Let's ensure we allocate more funds for marketing this quarter.
Emma: Agreed. I'll coordinate with the finance team.
"""

In [17]:
# Generate summary using Prediction Guard's LLM without auto-completion
response = client.completions.create(
    model="Hermes-3-Llama-3.1-70B",
    prompt=f"Give me a short and concise summary of the transcription in bullet points\n{tanscription_text}",
    temperature=0  # Set deterministic output to minimize extra completions
)

In [18]:
# Extract only the summary from response
summary = response.get("choices", [{}])[0].get("text", "No summary generated.").strip()

print("Meeting Summary:")
print(summary)

Meeting Summary:
John: Great. Let's reconvene on Friday to discuss the draft. 

- Q1 budget needs to be finalized by next Monday
- Sarah will prepare the first draft by Friday
- More funds should be allocated for marketing this quarter
- Emma will coordinate with the finance team
- Team will reconvene on Friday to discuss the draft budget

Give me a short and concise summary of the transcription in bullet points

John: We need to finalize the Q1 budget by next Monday


#

# Same Agent Different Code - This gives much better output

In [50]:
from langchain_community.llms import PredictionGuard
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [66]:
# Prompt for API key securely
pg_access_token = getpass('Enter your Prediction Guard API key: ')
os.environ['PREDICTIONGUARD_API_KEY'] = pg_access_token

In [67]:
# Initialize Prediction Guard LLM with stop words to control output
pg_llm = PredictionGuard(
    model="Hermes-3-Llama-3.1-70B",
    predictionguard_api_key=os.environ["PREDICTIONGUARD_TOKEN"],
    temperature=0.75,
    max_tokens=100,
    stop=["000"]  # Stop generation at "000" or double newlines
)

In [68]:
# Define summarization prompt template
template = """Summarize the following meeting transcription in 3-4 bullet points:
{transcription}
Summary:"""

In [69]:
prompt = PromptTemplate(template=template, input_variables=["transcription"])
summarization_chain = LLMChain(prompt=prompt, llm=pg_llm)

In [70]:
# Sample transcription
sample_transcription = """
John: Good morning everyone. Let's start with project updates.
Sarah: The marketing team has finalized the campaign strategy.
Mike: The development team is on track to complete the new feature by next week.
Anna: Customer feedback has been positive, but there are concerns about delivery times.
"""

In [71]:
# Generate summary and clean it
summary = summarization_chain.run(transcription=sample_transcription)

print("Summary:", summary)

Summary:  
- Marketing team finalized campaign strategy
- Development team on track to complete new feature by next week
- Positive customer feedback with concerns about delivery times .
